In [12]:
import os
import copy
import json
import time
import pandas as pd
import collections
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations, permutations, product

%matplotlib inline  

In [13]:
class BaseGraph:
    def __init__(self, inputs):
        self.inputs = inputs
        self.edge_list = None
        self.start_list = None
        self.end_list = None
        self.graph = nx.Graph()
    
    def create_edge_list(self):
        # create list of edges
        elist = []
        for i, val in enumerate(self.inputs['Edges']):
            e1 = val['Node1']
            e2 = val['Node2']
            elist.append((e1, e2, ))
            
        self.edge_list = elist
        
    def create_node_list(self):
        # Create list of starting and ending nodes
        start_list = []
        end_list = []
        for i, val in enumerate(self.inputs['Nodes']):
            if val['NodeType'] == 'Shaft':
                start_list.append(val['Id'])
            if val['NodeType'] == 'Vav':
                end_list.append(val['Id'])
                
        self.start_list = start_list
        self.end_list = end_list
        
    def add_edges_to_graph(self):
        self.graph.add_edges_from(self.edge_list)
        
    def print_graph(self):
        nx.draw(self.graph, with_labels=True, font_weight='bold')
        

In [14]:
class RouteOptions:
    def __init__(self, base_graph, source, targets):
        self.base_graph = base_graph
        self.source = source
        self.targets = targets
        self.individual_paths = None
        self.path_combinations = None
        self.graph_options = None
        
    def create_all_paths_to_each_target(self):
        paths = {}
        for target in self.targets:
            target_key = str(target)
            paths.update({target_key: []})
            #paths.update({target_key: {paths:[]}})
            for path in nx.all_simple_paths(self.base_graph, source=self.source, target=target):
                #print(paths[target_key])
                paths[target_key].append({"source": self.source, "target": target, "path": path})
                
        self.individual_paths = paths
        
    def create_all_path_combinations(self):
        # Create list of combinations of paths
        path_combinations = []
        for target in self.individual_paths:
            #print(list(range(0, len(paths[target])+1)))
            path_combinations.append(list(range(0, len(self.individual_paths[target]))))

        self.path_combinations = list(product(*path_combinations))
        #path_combinations
    
    def create_graph_options(self):
        # Create new graphs for each combination
        final_graphs = []

        # Loop for each path combination
        for path_combination in self.path_combinations:
            #print("")
            #print("path_combination: " + str(path_combination))
            new_graph = nx.Graph()

            # Loop for each target
            for target_number, target in enumerate(self.targets):
                #print("target_number: " + str(target_number))
                edge_list = []

                my_path = self.individual_paths[str(target)][path_combination[target_number]]['path']

                # Loop for each node in path
                for i in range(0, len(my_path)-1):
                    edge_list.append((my_path[i], my_path[i+1]))

                G1 = nx.Graph()
                G1.add_edges_from(edge_list)

                #print("edge_list: " + str(edge_list))

                if target_number > 0:
                    new_graph = nx.compose(G1, new_graph)
                else:
                    new_graph = G1

            final_graphs.append(new_graph)

        #print("")
        #print(final_graphs)
        self.graph_options = final_graphs
        
        print('number of graphs: ' + str(len(self.graph_options)))
        
    def print_graph_options(self):
        fig = plt.figure(figsize=(10, 5*(len(self.graph_options))))
        for i, F in enumerate(self.graph_options):
            ax = fig.add_subplot(len(self.graph_options),1,i+1)
            nx.draw(F, with_labels=True, font_weight='bold')

        plt.show()

In [15]:
t_start = time.time()

In [16]:
filename = 'Arch Model 4.json'

In [17]:
with open(os.path.join('..', 'models', filename)) as json_data:
#with open(os.path.join(filename)) as json_data:
    inputs = json.load(json_data)

#print(inputs)

In [18]:
# Run calclations to create base mesh
base_graph = BaseGraph(inputs)
base_graph.create_edge_list()
base_graph.create_node_list()
base_graph.add_edges_to_graph()
#base_graph.print_graph()

In [23]:
# Run calculations to create options

route_option_list = []
# Loop for each shaft option
#print('Number of shafts: ' + str(len(base_graph.start_list)))
#print('')

for start_node in base_graph.start_list:
    #print('Start node: ' + str(start_node))
    route_options = RouteOptions(base_graph.graph, start_node, base_graph.end_list)
    route_options.create_all_paths_to_each_target()
    route_options.create_all_path_combinations()
    route_options.create_graph_options()
    
    route_option_list.append(route_options)
    #route_options.print_graph_options()
    #print('')

number of graphs: 32


In [41]:
vav_nodes = [x for x in inputs['Nodes'] if x['NodeType'] == 'Vav']
space_Ids = [x['SpaceId'] for x in vav_nodes]
space_Ids
for x in space_Ids:
    for y in inputs['Spaces']:
        if y['UniqueId'] == x:
            print(x)
            print(y['Airflow'])

ca5699eb-5fbf-49e0-b030-42f100b1d06e-000eb891
0.0
ca5699eb-5fbf-49e0-b030-42f100b1d06e-000eb888
0.0
ca5699eb-5fbf-49e0-b030-42f100b1d06e-000eb88b
0.0
ca5699eb-5fbf-49e0-b030-42f100b1d06e-000eb88e
0.0
ca5699eb-5fbf-49e0-b030-42f100b1d06e-000eb897
0.0


In [42]:
#inputs['Spaces']

In [44]:
#route_option_list[0].graph_options[0].edges()
#inputs['Nodes']

In [20]:
t_stop = time.time()
print('time elapsed: ' + str(t_stop - t_start))

time elapsed: 6.06571364402771


In [21]:
# Print the edges for the final_graphs
#for final_graph in final_graphs:
#    print(final_graph.edges())

In [22]:
"""
# Find and remove closed loop solutions

# Determine closed loops
G = final_graphs[0]
DG = nx.DiGraph(G)
simple_cycles = list(nx.simple_cycles(DG))
simple_cycles = [x for x in simple_cycles if len(x) > 2]

for closed_loop in simple_cycles:
    # Loop for each node in path
    closed_loop_list = []
    closed_loop = simple_cycles[0]
    for i in range(0, len(closed_loop)-1):
        closed_loop_list.append((closed_loop[i], closed_loop[i+1]))
    closed_loop_list.insert(0, (0, 1))    

    # Remove graph if it matches cycle
    for i, final_graph in enumerate(final_graphs):
        #print(my_closed_loop)
        if final_graphs[i].edges() == closed_loop_list:
            final_graphs.remove(final_graph)
"""

'\n# Find and remove closed loop solutions\n\n# Determine closed loops\nG = final_graphs[0]\nDG = nx.DiGraph(G)\nsimple_cycles = list(nx.simple_cycles(DG))\nsimple_cycles = [x for x in simple_cycles if len(x) > 2]\n\nfor closed_loop in simple_cycles:\n    # Loop for each node in path\n    closed_loop_list = []\n    closed_loop = simple_cycles[0]\n    for i in range(0, len(closed_loop)-1):\n        closed_loop_list.append((closed_loop[i], closed_loop[i+1]))\n    closed_loop_list.insert(0, (0, 1))    \n\n    # Remove graph if it matches cycle\n    for i, final_graph in enumerate(final_graphs):\n        #print(my_closed_loop)\n        if final_graphs[i].edges() == closed_loop_list:\n            final_graphs.remove(final_graph)\n'